<a href="https://colab.research.google.com/github/subarnop/NLPseq2seq/blob/master/one2one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

Using TensorFlow backend.


In [0]:
#source text
data = """I'm beginning to feel like a Rap God, Rap God
          All my people from the front to the back nod, back nod
          Now who thinks their arms are long enough to slap box, slap box?
          They said I rap like a robot, so call me Rapbot"""


In [0]:
# integer encode text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded = tokenizer.texts_to_sequences([data])[0]

In [4]:

# determine the vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# create word -> word sequences
sequences = list()
for i in range(1, len(encoded)):
	sequence = encoded[i-1:i+1]
	sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))
# split into X and y elements
sequences = array(sequences)
X, y = sequences[:,0],sequences[:,1]
# one hot encode outputs
y = to_categorical(y, num_classes=vocab_size)
# define model
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=1))
model.add(LSTM(50))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())



Vocabulary Size: 35
Total Sequences: 45
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1, 10)             350       
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                12200     
_________________________________________________________________
dense_1 (Dense)              (None, 35)                1785      
Total params: 14,335
Trainable params: 14,335
Non-trainable params: 0
_________________________________________________________________
None


In [5]:
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(X, y, epochs=500, verbose=2)

Epoch 1/500
 - 1s - loss: 3.5552 - acc: 0.0000e+00
Epoch 2/500
 - 0s - loss: 3.5539 - acc: 0.0444
Epoch 3/500
 - 0s - loss: 3.5527 - acc: 0.0889
Epoch 4/500
 - 0s - loss: 3.5517 - acc: 0.1111
Epoch 5/500
 - 0s - loss: 3.5506 - acc: 0.0889
Epoch 6/500
 - 0s - loss: 3.5495 - acc: 0.1111
Epoch 7/500
 - 0s - loss: 3.5486 - acc: 0.1111
Epoch 8/500
 - 0s - loss: 3.5474 - acc: 0.1111
Epoch 9/500
 - 0s - loss: 3.5463 - acc: 0.1111
Epoch 10/500
 - 0s - loss: 3.5452 - acc: 0.1111
Epoch 11/500
 - 0s - loss: 3.5441 - acc: 0.1556
Epoch 12/500
 - 0s - loss: 3.5429 - acc: 0.1556
Epoch 13/500
 - 0s - loss: 3.5416 - acc: 0.1556
Epoch 14/500
 - 0s - loss: 3.5404 - acc: 0.1556
Epoch 15/500
 - 0s - loss: 3.5391 - acc: 0.1556
Epoch 16/500
 - 0s - loss: 3.5377 - acc: 0.1556
Epoch 17/500
 - 0s - loss: 3.5363 - acc: 0.1556
Epoch 18/500
 - 0s - loss: 3.5349 - acc: 0.1556
Epoch 19/500
 - 0s - loss: 3.5333 - acc: 0.1556
Epoch 20/500
 - 0s - loss: 3.5317 - acc: 0.1556
Epoch 21/500
 - 0s - loss: 3.5301 - acc: 0.15

In [0]:
# generate a sequence from the model
def generate_seq(model, tokenizer, seed_text, n_words):
	in_text, result = seed_text, seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		encoded = array(encoded)
		# predict a word in the vocabulary
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text, result = out_word, result + ' ' + out_word
	return result

In [7]:
# evaluate
print(generate_seq(model, tokenizer, 'They', 6))

They said i rap god rap god
